In [56]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types
from pyspark.sql.functions import col, unix_timestamp

In [5]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/02 19:06:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
spark.version

'3.5.5'

In [7]:
df = spark.read \
    .parquet('yellow_tripdata_2024-10.parquet')

In [10]:
df = df.repartition(4)

In [11]:
df.write.parquet('pq/')

In [13]:
import os

In [16]:
def get_average_parquet_size(directory):
    parquet_files = [f for f in os.listdir(directory) if f.endswith(".parquet")]
    
    if not parquet_files:
        return 0  # No Parquet files found
    
    total_size = sum(os.path.getsize(os.path.join(directory, f)) for f in parquet_files)
    average_size = total_size / len(parquet_files)
    
    return average_size

In [17]:
directory = "pq/"
average_size = get_average_parquet_size(directory)
print(f"Average size of Parquet files: {average_size / (1024 * 1024):.2f} MB")

Average size of Parquet files: 23.04 MB


In [18]:
df

DataFrame[VendorID: int, tpep_pickup_datetime: timestamp_ntz, tpep_dropoff_datetime: timestamp_ntz, passenger_count: bigint, trip_distance: double, RatecodeID: bigint, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, Airport_fee: double]

In [40]:
df_count = df \
    .withColumn("pickup_date", to_date(df.tpep_pickup_datetime)) 

In [41]:
df_count = df_count\
    .filter(df_count.pickup_date == "2024-10-15") \
    .count()
df_count

128893

In [43]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [60]:
df_trip = df \
    .withColumn('trip_duration', (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 3600) \
    .select('trip_duration').show()

[Stage 32:=============================>                            (1 + 1) / 2]

+--------------------+
|       trip_duration|
+--------------------+
|  0.5202777777777777|
|0.050555555555555555|
| 0.19777777777777777|
|  0.8927777777777778|
| 0.09944444444444445|
|  0.3713888888888889|
| 0.11777777777777777|
| 0.20666666666666667|
|  0.6480555555555556|
|  0.2452777777777778|
|  0.2747222222222222|
| 0.16944444444444445|
|  0.5566666666666666|
| 0.13361111111111112|
|0.050833333333333335|
|  0.6613888888888889|
|  0.1688888888888889|
|0.001944444444444...|
| 0.12861111111111112|
|              0.1575|
+--------------------+
only showing top 20 rows



AttributeError: 'NoneType' object has no attribute 'printSchema'

In [68]:
df_trip = df \
    .withColumn('trip_duration', (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 3600) \
    .select('trip_duration') \
    .orderBy(col('trip_duration').desc()) \
    .first()
df_trip

Row(trip_duration=162.61777777777777)